In [1]:
import os
os.chdir('..')

In [ ]:
import polars as pl

from config import BINANCE_DATA_DIR, TradeType
from generate.resample import resampled_kline
from util.time import convert_interval_to_timedelta

pl.Config.set_tbl_rows(30)

In [ ]:
spot_1m_kline_dir = BINANCE_DATA_DIR / "results_data" / "spot" / "klines" / "1m"
spot_1m_kline_dir

In [ ]:
symbol = 'BTCUSDT'
df_btc_1m = pl.read_parquet(spot_1m_kline_dir / f'{symbol}.pqt')
df_btc_1m.tail(5)

In [ ]:
def polars_calc_resample(df: pl.DataFrame, time_interval: str, resample_interval: str, offset_str: str) -> pl.DataFrame:
    """
    将 Polars K线 DataFrame, Resample 成带 offset 的大周期 K 线
    例如将 5m K线, Resample 成 5分钟偏移的小时线
    """
    time_interval = convert_interval_to_timedelta(time_interval)
    resample_interval = convert_interval_to_timedelta(resample_interval)
    offset = convert_interval_to_timedelta(offset_str)

    ldf = df.lazy()

    ldf = ldf.with_columns((pl.col('candle_begin_time') + time_interval).alias('candle_end_time'))
    ldf = ldf.group_by_dynamic("candle_begin_time", every=resample_interval, offset=offset).agg(
        pl.col("candle_begin_time").first().alias("candle_begin_time_real"),
        pl.col("candle_end_time").last(),
        pl.col("open").first(),
        pl.col("high").max(),
        pl.col("low").min(),
        pl.col("close").last(),
        pl.col("volume").sum(),
        pl.col("quote_volume").sum(),
        pl.col("trade_num").sum(),
        pl.col("taker_buy_base_asset_volume").sum(),
        pl.col("taker_buy_quote_asset_volume").sum(),
        pl.col("avg_price_1m").first(),
    )

    # 过滤掉 Resample 后，时间长度不足的 K 线
    ldf = ldf.filter((pl.col("candle_end_time") - pl.col("candle_begin_time_real")) == resample_interval)
    ldf = ldf.drop(["candle_begin_time_real", "candle_end_time"])
    return ldf.collect()

df_btc_1h = polars_calc_resample(df_btc_1m, '1m', '1h', '5m')
df_btc_1h.tail(5)

In [ ]:
resampled_kline(TradeType.spot, "BTCUSDT", "1h", "5m")

df_btc_1h = pl.read_parquet(
    "/home/lostleaf/crypto_data/binance_data/results_data/spot/resampled_klines/1h/35m/BTCUSDT.pqt"
)

df_btc_1h.tail()